In [ ]:
import os
os.getcwd()

In [ ]:
import sys
import os

#####################
# Import of utils.py functions
#####################
root_dir = os.getcwd()
sys.path.append('.')
from utils import loadFSL, FSLeyesServer

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################

# ???
# os.environ["DIPY_HOME"] = "/home/jovyan/Data"

#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries
##################
import fsl.wrappers
from fsl.wrappers import fslmaths, flirt, fast, bet, mcflirt, epi_reg, fslroi
from fsl.wrappers.misc import fslroi
import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri
import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

import pandas as pd
import numpy as np
import json
import subprocess

In [ ]:
# Cleanup previously defined data
os.system("rm -rf dataset/derivatives")

In [ ]:
%gui wx

In [ ]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

In [ ]:
# Creating derivative root and saving some useful paths
subject = "subject101410"

bids_root = op.join(root_dir, "dataset")
subject_root = op.join(bids_root, subject)

derivatives_root = op.join(bids_root, "derivatives")
os.makedirs(derivatives_root, exist_ok=True)
preproc_root = op.join(derivatives_root, "preprocessed_data")
os.makedirs(preproc_root, exist_ok=True)
subject_preproc_root = op.join(preproc_root, subject)
os.makedirs(subject_preproc_root, exist_ok=True)

anat_path = op.join(subject_root, "T1w", "T1w.nii.gz")

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(anat_path)

In [ ]:
# To display the commands and how to use the "bet" command
os.system("bet -h")

In [ ]:
# Skull stripping with "bet"
robust: bool = True

# Define the path where to save the file
os.makedirs(op.join(subject_preproc_root, "T1w"), exist_ok=True)
betted_brain_path = op.join(subject_preproc_root, "T1w", "T1w_skull.nii.gz")

# Apply "bet" and save the file
robust_text = "-R" if robust else ""
os.system(f"bet {anat_path} {betted_brain_path} -m {robust_text}")

# BET also creates a resulting Mask that is useful to display
resulting_mask = op.join(subject_preproc_root, "T1w", "T1w_skull_mask.nii.gz")

print("Skull stripping with BET succeeded!")

In [ ]:
# Display the Skull Stripped Image
fsleyesDisplay.load(resulting_mask)

In [ ]:
# Now we have to apply the mask to the image
def apply_fsl_math_approach(img_path, mask_path, masked_img_path):
    os.system(f'fslmaths {img_path} -mas {mask_path} {masked_img_path}')

apply_fsl_math_approach(anat_path, resulting_mask, betted_brain_path)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

In [ ]:
# Comparison between T1 and skull stripped
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)
fsleyesDisplay.load(anat_path)

In [ ]:
os.system("fast -h")

In [ ]:
# tissue segmentation
import time

start = time.time()
fast_output_path = op.join(subject_preproc_root, "T1w", "T1w_fasted")
fast_target = betted_brain_path 

# Quick cleanup
[os.remove(f) for f in glob.glob(op.join(subject_preproc_root, "T1w", '*fast*'))]

fast(imgs=[fast_target], out=fast_output_path, n_classes=3)
finish = time.time()
print(f"Segmentation succeded in {finish-start} seconds")

In [ ]:
print_dir_tree(preproc_root, max_depth=3)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)
fsleyesDisplay.load(glob.glob(op.join(subject_preproc_root, 'T1w','*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(subject_preproc_root, 'T1w','*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(subject_preproc_root, 'T1w','*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## I have checked and made sure everything works up to here, Check what is going on later

In [ ]:
# checking what does RL and LR mean
func_LR_path = op.join(subject_root, 'fMRI', 'tfMRI_MOTOR_LR','tfMRI_MOTOR_LR.nii')
func_RL_path = op.join(subject_root, 'fMRI', 'tfMRI_MOTOR_RL','tfMRI_MOTOR_RL.nii')

# creating folder

func_preproc_path = op.join(subject_preproc_root, 'fMRI')
LR_preproc_path=op.join(func_preproc_path,'tfMRI_MOTOR_LR')
RL_preproc_path=op.join(func_preproc_path,'tfMRI_MOTOR_RL')

In [ ]:
os.makedirs(op.join(subject_preproc_root,'fMRI'))
os.makedirs(op.join(func_preproc_path,'tfMRI_MOTOR_LR'))
os.makedirs(op.join(func_preproc_path,'tfMRI_MOTOR_RL'))

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(func_LR_path) # dimension4 284

In [ ]:
# scaling variance to one
import numpy as np
temp = nib.load(func_LR_path)
LR_func_data= temp.get_fdata()
std_LR=LR_func_data.std(axis=(0,1,2))
LR_norm = LR_func_data / std_LR
LR_norm_img = nib.Nifti1Image(LR_norm.astype(np.uint8), temp.affine, temp.header)
# LR_path = op.join(preproc_root, 'fMRI','tfMRI_MOTOR_LR_norm.nii')

nib.save(LR_norm_img, op.join(LR_preproc_path,'tfMRI_MOTOR_LR_norm.nii'))

temp = nib.load(func_RL_path)
RL_func_data= temp.get_fdata()
std_RL=RL_func_data.std(axis=(0,1,2))
RL_norm = RL_func_data / std_RL
RL_norm_img = nib.Nifti1Image(RL_norm.astype(np.uint8), temp.affine, temp.header)
# RL_path = op.join(preproc_root, 'fMRI','tfMRI_MOTOR_RL_norm.nii')
nib.save(RL_norm_img, op.join(RL_preproc_path,'tfMRI_MOTOR_RL_norm.nii'))

In [ ]:
# concatenating runs
print(LR_norm.std())

In [ ]:
merged_path=op.join(func_preproc_path, 'tfMRI_MOTOR_merged')
subprocess.run(['fslmerge', '-t', merged_path, 
                    op.join(LR_preproc_path,'tfMRI_MOTOR_LR_norm.nii'), 
                    op.join(RL_preproc_path,'tfMRI_MOTOR_RL_norm.nii')])

In [ ]:
# Overview
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(merged_path)

In [ ]:
# We use as reference middle volume of first run RL, DELETE .MAT FILES BETWEEN RUNS
reference_path=op.join(func_preproc_path, 'ftMRI_MOTOR_reference.nii.gz')
fslroi(merged_path, reference_path ,str(423),str(1))
motion_corrected_path=op.join(func_preproc_path, 'tfMRI_MOTOR_moco')
mcflirt(infile = merged_path, o =motion_corrected_path, reffile=reference_path ,plots=True, report=True, dof=6, mats=True)

In [ ]:
#Overview, 2mm resolution
fsleyesDisplay.load(op.join(func_preproc_path, 'tfMRI_MOTOR_moco.nii.gz'))

In [ ]:
#Calculating framewise displacement
def load_mot_params_fsl_6_dof(path):
    return pd.read_csv(path, sep='  ', header=None, 
            engine='python', names=['Rotation x', 'Rotation y', 'Rotation z','Translation x', 'Translation y', 'Translation z'])

mot_params = load_mot_params_fsl_6_dof(op.join(func_preproc_path, 'tfMRI_MOTOR_moco.par'))



In [ ]:
import matplotlib.pyplot as plt
def compute_FD_power(mot_params):
    framewise_diff = mot_params.diff().iloc[1:]

    rot_params = framewise_diff[['Rotation x', 'Rotation y', 'Rotation z']]
    # Estimating displacement on a 50mm radius sphere
    # To know this one, we can remember the definition of the radian!
    # Indeed, let the radian be theta, the arc length be s and the radius be r.
    # Then theta = s / r
    # We want to determine here s, for a sphere of 50mm radius and knowing theta. Easy enough!
    
    # Another way to think about it is through the line integral along the circle.
    # Integrating from 0 to theta with radius 50 will give you, unsurprisingly, r0 theta.
    converted_rots = rot_params*50
    trans_params = framewise_diff[['Translation x', 'Translation y', 'Translation z']]
    fd = converted_rots.abs().sum(axis=1) + trans_params.abs().sum(axis=1)
    return fd

fd = compute_FD_power(mot_params).to_numpy()
plt.plot(list(range(1, fd.size+1)), fd)
plt.xlabel('Volume')
plt.ylabel('FD displacement (mm)')
#plt.hlines(threshold, 0, 370,colors='black', linestyles='dashed', label='FD threshold')
plt.legend()
plt.show()

In [ ]:
coregister_path=op.join(func_preproc_path, 'tfMRI_MOTOR_coregistered')
white_matter_segmentation=op.join(subject_preproc_root,  'T1w','T1w_fasted_pve_2.nii.gz')
use_single_vol= True
if use_single_vol:

    subprocess.run(['epi_reg','--epi={}'.format(reference_path), '--t1={}'.format(anat_path), '--t1brain={}'.format(betted_brain_path), '--out={}'.format(coregister_path),'--wmseg={}'.format(white_matter_segmentation)])
else:
    subprocess.run(['epi_reg','--epi={}'.format(motion_corrected_path), '--t1={}'.format(anat_path), '--t1brain={}'.format(betted_brain_path), '--out={}'.format(coregister_path),'--wmseg={}'.format(white_matter_segmentation)])

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(coregister_path)
fsleyesDisplay.load(anat_path)

In [ ]:
# g
FMWH=4

cmd = 'fslmaths {} -s {} {}_smoothed-4mm'.format(motion_corrected_path, FMWH/2.3548, motion_corrected_path)
subprocess.run(['fslmaths',motion_corrected_path, '-s', str(FMWH/2.3548), '{}_smoothed-4mm'.format(motion_corrected_path)])

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(func_preproc_path, 'tfMRI_MOTOR_moco_smoothed-4mm')) # check error